# 필요한 라이브러리 Import

In [7]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
kiwi=Kiwi()
stopwords =Stopwords()

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import tensorflow as tf

In [9]:
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [11]:
import tensorflow as tf
tf.keras.backend.clear_session()

# Part | 데이터 load

In [12]:
data = pd.read_csv('~/aiffel/dktc/data_forder/Final_Train.csv')

In [13]:
class TextProcessor:
    def __init__(self):
        self.kiwi = Kiwi(typos='basic_with_continual_and_lengthening')
        self.stopwords = Stopwords()
        
    # 데이터 정제 (불필요한 문자, 특수 문자, 이메일, URL, 자음/모음 등 제거)
    def clean_text(self, text):
        # E-mail 제거
        text = re.sub(r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', '', text)
        # URL 제거
        text = re.sub(r'(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text)
        # 한글 자음, 모음 제거
        text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ]+)', '', text)
        # 알파벳, 숫자 제거
        text = re.sub(r'([a-zA-Z0-9]+)', '', text)
        # HTML 태그 제거
        text = re.sub(r'<[^>]*>', '', text)
        # 특수 기호 제거
        text = re.sub(r'[^\w\s]', '', text)
        # 한글과 공백을 제외한 모든 문자 제거
        text = re.sub(r'[^가-힣\s]', '', text)
        # 다중 공백을 단일 공백으로
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    # 형태소 분석 및 불용어 제거
    def analyze_morphs(self, text):
        # 형태소 분석 수행 및 불용어 제거
        tokens = [(token.form,token.tag )for token in self.kiwi.tokenize(text, stopwords=self.stopwords)]
        return tokens
    
    # 'EF' 태그가 있는 형태소 제거
    def filter_ef_tag(self, tokens):
        # 'EF' 품사 태그가 있는 형태소 제거
        filtered_tokens = [token[0] for token in tokens if token[1] != 'EF']
        return filtered_tokens

    # 추가적인 단어 필터링
    def filter_specific_words(self, tokens):
        words_to_remove = ['ㅂ니다', '안', '야', '너', '키키', '키', '거', 'ㄴ가요', '습니다']
        filtered_tokens = [token for token in tokens if token not in words_to_remove]
        return filtered_tokens

    # 전체 과정 통합 (형태소 분석, EF 품사 및 불용어 제거)
    def process(self, text):
        cleaned_text = self.clean_text(text)
        tokens = self.analyze_morphs(cleaned_text)
        tokens_without_ef = self.filter_ef_tag(tokens)
        filtered_tokens = self.filter_specific_words(tokens_without_ef)
        return filtered_tokens

In [ ]:
processor = TextProcessor()

data['ppc'] = data['conversation'].apply(lambda text: processor.process(text))

In [ ]:
data[['conversation','ppc']].head(50)

# Part | 빈도수 시각화

In [ ]:
# 나눔고딕 폰트 설치
!apt-get install -y fonts-nanum

# 설치된 나눔고딕 폰트를 matplotlib에 적용하기 위한 코드입니다.
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# 나눔고딕 폰트 경로를 가져와서 matplotlib에 적용
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)
plt.rc('font', family=fontprop.get_name())  # 폰트 설정

In [ ]:
# 각 클래스별로 형태소 빈도 계산 및 시각화 준비
def visualize_class_wordcloud(df):
    classes = df['class'].unique()
    
    for class_name in classes:
        # 해당 클래스에 속하는 문장의 형태소 추출
        class_morphs = df[df['class'] == class_name]['ppc'].sum()  # 모든 형태소 리스트를 합침
        
        # 형태소 빈도 계산
        word_freq = Counter(class_morphs)
        
        # Word Cloud 생성
        wordcloud = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', 
                              width=800, height=400, background_color='white').generate_from_frequencies(word_freq)
        
        # Word Cloud 시각화
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Word Cloud for {class_name}')
        plt.show()

# WordCloud 시각화 실행
visualize_class_wordcloud(data)

In [ ]:
# 각 클래스별로 형태소 빈도 계산 및 시각화 준비
def visualize_class_barplot(df):
    classes = df['class'].unique()
    
    for class_name in classes:
        # 해당 클래스에 속하는 문장의 형태소 추출
        class_morphs = df[df['class'] == class_name]['ppc'].sum()  # 모든 형태소 리스트를 합침
        
        # 형태소 빈도 계산
        word_freq = Counter(class_morphs)
        
        # 상위 20개의 형태소만 추출
        most_common_20 = word_freq.most_common(20)
        
        # 형태소와 빈도 리스트 분리
        words, counts = zip(*most_common_20)
        
        # 바 그래프 생성
        plt.figure(figsize=(10, 8))
        plt.barh(words, counts, color='skyblue')
        plt.xlabel('빈도수')
        plt.ylabel('형태소')
        plt.title(f'{class_name} - 상위 500개 형태소 빈도수', fontproperties=fontprop)
        plt.gca().invert_yaxis()  # 가장 빈도가 높은 단어가 상단에 오도록
        plt.show()

# 바 그래프 시각화 실행
visualize_class_barplot(data)

In [ ]:
df = data.copy()

# 레이블 인코딩
label_mapping = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화' : 4
}
df['class'] = df['class'].map(label_mapping)

In [ ]:
# 토크나이저 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['ppc'])

# 시퀀스 인코딩
sequences = tokenizer.texts_to_sequences(df['ppc'])

In [ ]:
sequence_lengths = [len(seq) for seq in sequences]

plt.figure(figsize=(8, 6))
plt.boxplot(sequence_lengths, vert=False)
plt.title('Boxplot of Sequence Lengths')
plt.xlabel('Sequence Length')
plt.show()

In [ ]:
# 최대 시퀀스 길이 결정 (% 로 결정)
max_seq_len = 40

In [ ]:
# 패딩 적용
X = pad_sequences(sequences, maxlen=max_seq_len, padding='pre')

# 레이블 준비
y = df['class'].values